# Dashboard Liderazgo

In [ ]:
import os

from_drive = True  # same flag you use everywhere

if os.environ.get("ATLAS_BOOTSTRAPPED") != "1":
    # ---------- GIT ON COLAB ONLY ----------
    try:
        from google.colab import userdata

        git_token = userdata.get('gitToken')
        git_user = userdata.get('gitUser')
        git_url = f'https://{git_token}@github.com/rene-aum/Atlas.git'
        branch_to_pull = 'dev'

        os.chdir('/content')

        if not os.path.isdir('Atlas'):
            !git clone {git_url}

        %cd Atlas
        !git fetch origin {branch_to_pull}
        !git checkout {branch_to_pull}
        !git pull origin {branch_to_pull}

        !pip install -r PipelinesConsumo/src/requirements.txt
        %cd PipelinesConsumo

    except Exception as e:
        print(e)
        print('Running in other environment not colab probably!')

    # ---------- DRIVE + SHEETS ----------
    if from_drive:
        from pydrive2.auth import GoogleAuth
        from pydrive2.drive import GoogleDrive
        from google.colab import auth
        from oauth2client.client import GoogleCredentials
        import gspread
        from google.auth import default
        from gspread_dataframe import set_with_dataframe
        import gdown

        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        drive = GoogleDrive(gauth)

        creds, _ = default()
        gc = gspread.authorize(creds)

    os.environ["ATLAS_BOOTSTRAPPED"] = "1"
else:
    print("Bootstrap already done, assuming orchestrator ran it.")

In [ ]:
import sys
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import pytz
# from matplotlib.ticker import FuncFormatter
from datetime import datetime, timedelta
import warnings
import sys
sys.path.append('..')
sys.path.append('../..')
from utils.utils import (get_dates_dataframe,
                       add_year_week,
                       custom_read,
                       process_columns)
from PipelinesConsumo.src.rawAtlas import RawAtlas
from PipelinesConsumo.src.processedAtlas import ProcessedAtlas
from src.transformed import Transformed
from utils.drive_toolbox import(from_drive_to_local,
                             get_last_modification_date_drive,
                             create_sheets_in_drive_folder,
                             update_sheets_in_drive_folder,
                             read_from_google_sheets,
                             list_file_ids_for_drive_folder,
                             create_csv_file_in_drive_folder,
                             write_csv_to_drive,
                             read_csv_from_drive)
from utils.utils import get_dates_dataframe
from src.constants import (atlas_raw_output_folder_id,
                           atlas_consumo_output_folder_id,
                           consumo_sheets_ids_dict,
                           data_source_folder_id,
                           raw_output_ids,
                           folder_id_bauto_gabo,
                           id_reporte_ventas,
                           )


warnings.filterwarnings('ignore')



In [ ]:
output_sheet_id = '1ET9_koU3bMWBJv8YyXV6l_yfGF9qLpraKXqUJOdALiY'

## Load

In [ ]:
datedf = get_dates_dataframe(start='2025-01-01').assign(date = lambda x: x.date.dt.strftime('%Y-%m-%d'))

In [ ]:
# consumo_sheets_ids_dict
visitas_unicas = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcVisitasUnicas'])
funnel_vendedor = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcAdobeFunnelVendedorTotal'])
funnel_comprador = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcAdobeFunnelCompradorTotal'])
publicaciones = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcPublicaciones'])
pedidos = read_from_google_sheets(gc,consumo_sheets_ids_dict['AcPedidos'])


## Vendedor

In [ ]:
sum_cols_vendedor_visits = ['fv_visitas_unicas_landing_vendedor']
vendedor0 = (visitas_unicas
 [lambda x: pd.to_datetime(x.date).dt.year>=2025]
 .groupby(['date'])[sum_cols_vendedor_visits].sum()
 )


In [ ]:
sum_cols_vendedor = funnel_vendedor.filter(like='fv_').columns
rename_dict_vendedor = {'fv_app_click_start':'fv_0_app_click_start',
                        'fv_app_page_visit':'fv_1_marca_modelo',
                        'fv_app_step_2':'fv_2_version',
                        'fv_app_step_3':'fv_3_tenemos_tu_version',
                        'fv_app_step_4':'fv_4_precio',
                        'fv_app_step_6':'fv_5_datos_complementarios',
                        'fv_app_step_8':'fv_6_espacio_automarket',
                        'fv_app_step_9':'fv_7_fotografias',
                        'fv_app_step_10':'fv_8_exito_publicacion'
                        }
vendedor1 = (funnel_vendedor
    .assign(year_month=lambda x:pd.to_datetime( x['date']).dt.strftime('%Y-%m'),
            flag_registered = lambda x: x.id_am.notna().astype('Int64')
            )
    [lambda x: pd.to_datetime(x.date).dt.year>=2025]
    .groupby(['date'])[sum_cols_vendedor].sum()
    .rename(columns=rename_dict_vendedor)
    
)


In [ ]:

vendedor2 =(pedidos
 .assign(year_month=lambda x:pd.to_datetime( x['fecha_de_creacion']).dt.strftime('%Y-%m'))
 [lambda x:pd.to_datetime(x.fecha_de_creacion).dt.year>=2025]
  .rename(columns = {'fecha_de_creacion':'date'})
 .groupby(['date'])['sf_order_id'].nunique()
 .rename('pedidos')
 .to_frame()
 )


In [ ]:
vendedor3 = (publicaciones
 [lambda x: x.published_at.notna()]
 .assign(year_month=lambda x:pd.to_datetime( x['published_at']).dt.strftime('%Y-%m'),)
 [lambda x: pd.to_datetime(x.published_at).dt.year>=2025]
  .rename(columns={'published_at':'date'})
 .groupby(['date'])['sku'].nunique()
 .rename('publicaciones')
 .to_frame()
#  .T
 
 )


## Comprador

In [ ]:
visitas_unicas.head()


In [ ]:
sum_cols_comprador_visits = ['fc_visitas_unicas_landing_comprador','fc_visitas_unicas_pdp','fc_visitas_unicas_plp']
comprador0 = (visitas_unicas
 .assign(year_month=lambda x:pd.to_datetime( x['date']).dt.strftime('%Y-%m'),
            )
 [lambda x: pd.to_datetime(x.date).dt.year>=2025]
 .groupby(['date'])[sum_cols_comprador_visits].sum()
 )
sum_cols_comprador = funnel_comprador.filter(like='fc_').columns
rename_dict_comprador = {'fc_app_click_start':'fc_0_app_click_start',
                        'fc_app_page_visit':'fc_1_onboarding',
                        'fc_app_completed':'fc_2_apartado_exitoso',
                        }


In [ ]:
comprador1 = (funnel_comprador
    .assign(year_month=lambda x:pd.to_datetime( x['date']).dt.strftime('%Y-%m'),
            flag_registered = lambda x: x.id_am.notna().astype('Int64')
            )
    [lambda x: pd.to_datetime(x.date).dt.year>=2025]
    .groupby(['date'])[sum_cols_comprador].sum()
    .rename(columns=rename_dict_comprador)
    # .T
)

In [ ]:
comprador2 =(pedidos
 .assign(year_month=lambda x:pd.to_datetime( x['fecha_de_creacion']).dt.strftime('%Y-%m'))
 [lambda x:pd.to_datetime(x.fecha_de_creacion).dt.year>=2025]
             .rename(columns = {'fecha_de_creacion':'date'})
 .groupby(['date','multiapartado'])['sf_order_id'].nunique()
.unstack('multiapartado')
.rename(columns={0:'apartados_unicos',
               1:'multiapartados'})
.fillna(0)
 )

## Master


In [ ]:

master = (datedf
          .merge(vendedor0,on='date',how='left')
          .merge(vendedor1,on='date',how='left')
          .merge(vendedor2,on='date',how='left')
          .merge(vendedor3,on='date',how='left')
          .merge(comprador0,on='date',how='left')
          .merge(comprador1,on='date',how='left')
          .merge(comprador2,on='date',how='left')
          .fillna(0)
          .assign(year = lambda x:pd.to_datetime(x.date).dt.year,
                  month = lambda x:pd.to_datetime(x.date).dt.month,
                  year_month=lambda x:pd.to_datetime( x['date']).dt.strftime('%Y-%m'),
                  year_week = lambda x:pd.to_datetime( x['date']).dt.strftime('%Y-%U'),
                  day_of_week = lambda x:pd.to_datetime( x['date']).dt.strftime('%A'))
          )

## Final

In [ ]:
period_column = 'year_month'
vendedor_columns = master.filter(regex=f'{period_column}|^fv_|pedidos|publicaciones').columns
comprador_columns = master.filter(regex=f'{period_column}|^fc_|pedidos|apartados_unicos|multiapartados').columns
final_vendedor = (master
 [vendedor_columns]
 .groupby(['year_month'])
 .sum()
 .T
 )
final_comprador = (master
                   [comprador_columns]
                   .groupby(['year_month'])
                   .sum()
                   .T
                   )

## write 

In [ ]:
output_id_funnel_phyg = '1ET9_koU3bMWBJv8YyXV6l_yfGF9qLpraKXqUJOdALiY'
update_sheets_in_drive_folder(gc,output_id_funnel_phyg,'Master',master)

## free memory for orchestrator

In [ ]:
vars_to_del = ['vendedor0','vendedor1','vendedor2','vendedor3','comprador0','comprador1','comprador2',
               'visitas_unicas','funnel_comprador','funnel_vendedor','publicaciones','pedidos','master']
for v in vars_to_del:
    try:
        del globals()[v]
    except Exception as e:
        print(f"could not delete var {v}: {e}")

In [ ]:
import gc as gcol
gcol.collect()